In [1]:
from scipy.spatial import KDTree
from geopy.distance import geodesic
import pandas as pd

In [2]:
gh_df = (
    pd.read_csv("geohash_traffic_density_pt_15clusters.csv", usecols=['GEOHASH', 'LATITUDE', 'LONGITUDE'])
)

# Reading ISPARK data
ispark_df = pd.read_csv("datasets/11_tr_ist_ispark_parking/tr_ist_ispark_list.csv", usecols=['parkName', 'lat', 'lng', 'capacity'])

In [3]:
site_dict = (
    ispark_df.set_index('parkName')
    .T.to_dict()
)

In [4]:
# Unique list of coordinates
coordinates = gh_df[['LATITUDE', 'LONGITUDE']].drop_duplicates().values
coordinates

array([[41.080627, 28.811646],
       [40.987244, 29.108276],
       [41.003723, 29.09729 ],
       ...,
       [40.976257, 29.229126],
       [41.009216, 27.98767 ],
       [41.020203, 27.998657]])

In [5]:
def find_nearest_coordinates(
        target_lat: float,
        target_lon: float,
        coordinates: list[list[float, float]]
        ) -> list[tuple[tuple[float, float], float]]:
    """
    Finds the nearest coordinates to a target latitude and longitude.

    Args:
        target_lat (float): The latitude of the target location.
        target_lon (float): The longitude of the target location.
        coordinates (List[Tuple[float, float]]): A list of tuples representing the coordinates to search from.

    Returns:
        List[Tuple[Tuple[float, float], float]]: A list of tuples containing the nearest coordinates and their distances in kilometers.
    """
    target_coords = (target_lat, target_lon)
    
    # Build a KDTree for efficient nearest neighbor search
    tree = KDTree(coordinates)

    # Calculate the search radius in degrees
    radius_deg = 1 / 111.0  # 1 km radius in degrees

    # Query the KDTree for indices of points within the search radius
    indices = tree.query_ball_point([(target_lat, target_lon)], radius_deg)

    # Calculate the distance between target coordinates and nearest coordinates
    nearest_coordinates = [(coordinates[i], geodesic(target_coords, coordinates[i]).km) for i in indices[0]]

    # Sort the nearest coordinates by distance
    nearest_coordinates.sort(key=lambda x: x[1])  # Sort by distance

    return nearest_coordinates

In [6]:
ispark_list = []
new_list = []

for site in site_dict:

    target_lat = site_dict[site]['lat']
    target_lon = site_dict[site]['lng']

    nearest_coords = find_nearest_coordinates(target_lat, target_lon, coordinates)

    for i, (coords, distance) in enumerate(nearest_coords):

        gh = gh_df.loc[gh_df['LATITUDE'].eq(coords[0]) & gh_df['LONGITUDE'].eq(coords[1]), 'GEOHASH'].values[0]

        # Creating two dataframe which will later be merged rows wise for visualization purpose.
        ispark_list.append({
            "type" : "ispark_parking",
            "name" : site,
            "latitude" : target_lat,
            "longitude" : target_lon,
            "nearest" : site
        })

        new_list.append({
            "type" : "geohash",
            "name" : gh,
            "latitude" : coords[0],
            "longitude" : coords[1],
            "nearest" : site
        })

In [7]:
df = (
    pd.concat([pd.DataFrame(new_list), pd.DataFrame(ispark_list)], ignore_index=True)
    .drop_duplicates(ignore_index=True)
)

df

,type,name,latitude,longitude,nearest
0,geohash,sxk96u,41.025696,28.954468,1293 Yavuz Selim Otomatik Otoparkı
1,geohash,sxk96g,41.020203,28.954468,1293 Yavuz Selim Otomatik Otoparkı
2,geohash,sxk96v,41.031190,28.954468,1293 Yavuz Selim Otomatik Otoparkı
3,geohash,sxk96f,41.014710,28.954468,1293 Yavuz Selim Otomatik Otoparkı
4,geohash,sxk93x,41.042175,28.899536,15 Temmuz Demokrasi Otogarı
...,...,...,...,...,...
2844,ispark_parking,Yıldızposta Caddesi 9,41.066400,29.005900,Yıldızposta Caddesi 9
2845,ispark_parking,Zeynep Kamil Hastanesi Önü 1,41.015700,29.025200,Zeynep Kamil Hastanesi Önü 1
2846,ispark_parking,Zeytinburnu 58. Bulvar 2,40.993400,28.903800,Zeytinburnu 58. Bulvar 2
2847,ispark_parking,Zeytinburnu 58. Bulvar 6,40.989300,28.903800,Zeytinburnu 58. Bulvar 6


In [8]:
# Some GEOHASH has up to 42 ISPARKS within its 1km radius
(
    df.query("type=='geohash'")
    .groupby('name')['nearest'].nunique()
    .nlargest(10)
)

name
sxk9e9    42
sxk9eb    42
sxk9e8    38
sxk97z    33
sxk96d    32
sxk96g    31
sxk97x    29
sxk96e    27
sxk9ec    27
sxk9sn    26
Name: nearest, dtype: int64

In [10]:
df.to_csv("datasets/13_gh_proximities/gh_nearby_isparks.csv", index=False)